<a href="https://colab.research.google.com/github/OscarCabreraRodriguez/AnomalyDetectionSwat2019/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [2]:
# mount the data from google drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df = pd.read_excel('/content/gdrive/My Drive/SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'], index_col = 'GMT +0')
df = df.rename(columns=lambda x: x.strip())

In [4]:
 #active 1 inactive 0
#  'LS 201', 'LS 202','LSL 203','LSLL 203'  ,'LS 401','LSH 601','LSH 602','LSH 603','LSL 601','LSL 602','LSL 603'
df['LS 201']=np.where(df['LS 201'] == 'Active', 1, 0)
df['LS 202']=np.where(df['LS 202'] == 'Active', 1, 0)
df['LSL 203']=np.where(df['LSL 203'] == 'Inactive', 0, 1)
df['LSLL 203']=np.where(df['LSLL 203'] == 'Active', 1, 0)
df['LS 401']=np.where(df['LS 401'] == 'Active', 1, 0)
df['LSH 601']=np.where(df['LSH 601'] == 'Active', 1, 0)
df['LSH 602']=np.where(df['LSH 602'] == 'Active', 1, 0)
df['LSH 603']=np.where(df['LSH 603'] == 'Active', 1, 0)
df['LSL 601']=np.where(df['LSL 601'] == 'Active', 1, 0)
df['LSL 602']=np.where(df['LSL 602'] == 'Active', 1, 0)
df['LSL 603']=np.where(df['LSL 603'] == 'Active', 1, 0)
df_orginal = df.copy()


## Remove constant features

In [5]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(df_orginal)  # fit finds the features with zero variance

VarianceThreshold(threshold=0)

In [6]:
# get_support is a boolean vector that indicates which features are retained
# if we sum over get_support, we get the number of features that are not constant
# constant features
const_features = [x for x in df_orginal.columns if x not in df_orginal.columns[sel.get_support()]]
print(f"Number of constant feature {len(const_features)}")
print(const_features)

Number of constant feature 33
['P102 Status', 'LS 201', 'LS 202', 'LSL 203', 'LSLL 203', 'P2_STATE', 'P201 Status', 'P202 Status', 'P204 Status', 'P206 Status', 'P207 Status', 'P208 Status', 'P302 Status', 'AIT 401', 'LS 401', 'P4_STATE', 'P402 Status', 'P403 Status', 'P404 Status', 'MV 502', 'MV 503', 'MV 504', 'P5_STATE', 'P501 Status', 'P502 Status', 'LSH 602', 'LSH 603', 'LSL 601', 'LSL 602', 'LSL 603', 'P6 STATE', 'P602 Status', 'P603 Status']


In [7]:
df = df_orginal[df_orginal.columns[sel.get_support()]]
print(f"Number of feature before removing constant features {df_orginal.shape[1]}")
print(f"Number of feature after removing constant features {df.shape[1]}")

Number of feature before removing constant features 77
Number of feature after removing constant features 44


## Removing quasi-constant features

In [8]:
sel1 = VarianceThreshold(threshold=0.01)
sel1.fit(df)  # fit finds the features with zero variance
sum(sel1.get_support())
const_features2 = [x for x in df.columns if x not in df.columns[sel1.get_support()]]
print(len(const_features2))

11


In [9]:
df2 = df[df.columns[sel1.get_support()]]
print(f"Number of feature before removing quasi-constant features {df.shape[1]}")
print(f"Number of feature after removing quasi-constant features {df2.shape[1]}")

Number of feature before removing quasi-constant features 44
Number of feature after removing quasi-constant features 33


We can see that 33 features are constants and 11 feature are almost constant (quasi-constant). This means that 11 variables show predominantly one value for ~99% the observations.

In [10]:
print(len(df.columns)/len(df_orginal.columns))
print((len(df.columns)-len(df2.columns))/len(df_orginal.columns))
print(len(df2.columns)/len(df_orginal.columns))

0.5714285714285714
0.14285714285714285
0.42857142857142855


We can see that 57% of the features are constant and 14% are quasi-constant after removing both of them we keep on 42% only (Is this CORRECT!!)

#### Other methods for features selection
**SelectPercentile, SelectKBeast**

# Data Reading

In [11]:
#Data description - classfiying the coloumns in to sessor and actuator 
 #In SWaT, the sensor data can be continuous, while pumps only have 2 statuses (2 = open and 1 = closed), 
 #active 1 inactive 0

#  'LS 201', 'LS 202','LSL 203','LSLL 203'  ,'LS 401','LSH 601','LSH 602','LSH 603','LSL 601','LSL 602','LSL 603'
 
sensor_columns = ['FIT 101','LIT 101','AIT 201','AIT 202','AIT 203','FIT 201','AIT 301','AIT 302','AIT 303','DPIT 301','FIT 301','LIT 301','AIT 402','FIT 401','LIT 401',

'AIT 501','AIT 502','AIT 503','AIT 504','FIT 501','FIT 502','FIT 503','FIT 504','PIT 501','PIT 502',	'PIT 503','FIT 601'
]
actuator_columns = ['MV 101','P1_STATE','P101 Status', 'P102 Status', 'LS 201', 'LS 202','LSL 203','LSLL 203','MV201','P2_STATE','P201 Status','P202 Status','P203 Status','P204 Status','P205 Status','P206 Status','P207 Status','P208 Status',
                    'MV 301','MV 302','MV 303','MV 304','P3_STATE','P301 Status','P302 Status','AIT 401',
'LS 401','P4_STATE','P401 Status','P402 Status','P403 Status','P404 Status','UV401','MV 501','MV 502','MV 503','MV 504','P5_STATE','P501 Status','P502 Status',
'LSH 601','LSH 602','LSH 603','LSL 601','LSL 602','LSL 603','P6 STATE','P601 Status','P602 Status','P603 Status'

]
timestamp_col = 'Timestamp' 
label_col = 'Normal/Attack'

In [12]:
ts_with_attackes = ["FIT 401", "LIT 301", "P601 Status","MV201", "P101 Status", "MV 501", "P301 Status"]

In [13]:
set(ts_with_attackes)-set([x for x in ts_with_attackes if x in df.columns])

set()

In [14]:
set(ts_with_attackes)-set([x for x in ts_with_attackes if x in df2.columns])

{'FIT 401'}

one of the attacked sensors is quasi-constant

In [15]:
actuator_columns = list(set(actuator_columns).intersection(df2.columns))
sensor_columns = list(set(sensor_columns).intersection(df2.columns))

# Preprocessing

In [16]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [17]:
def preprocess(df):
  df = df.copy()
 #OneHotEncode categorical data
  encoder = OneHotEncoder(sparse = False)
  enc_df = pd.DataFrame(encoder.fit_transform(df[actuator_columns]))
  enc_df.index = df.index
  df.drop(actuator_columns, axis='columns', inplace=True)
  
  # Normalize numerical data
  minmax_scale = pd.DataFrame(MinMaxScaler().fit_transform(df[sensor_columns]))
  minmax_scale.index = df.index 
  df.drop(sensor_columns, axis='columns', inplace=True)
  set_val = pd.concat([enc_df,df, minmax_scale], axis=1, ignore_index=True) 
  set_val = set_val.reindex(df.index)
  return set_val 

In [18]:
processed_select_df = preprocess(df)
processed_select_df2 = preprocess(df2)
processed_df = preprocess(df_orginal)

In [19]:
import plotly.graph_objects as go
def plot_anomalies(isf_dataset, anomalies, title):
  b1 = go.Scatter(x=isf_dataset.index.astype(str),
                  y=isf_dataset['Anomaly'],
                  name="Dataset",
                  mode='markers'
                )
  b2 = go.Scatter(x=anomalies.index.astype(str),
                  y=anomalies['Anomaly'],
                  name="Anomalies",
                  mode='markers',
                  marker=dict(color='red', size=6,
                              line=dict(color='red', width=1))
                )

  data = [b1, b2]

  fig = go.Figure(data=data)
  fig.show()

# IsolationForest



Isolation Forests
Isolation Forests is an unsupervised learning algorithm that identifies anomalies by isolating outliers in the data based on the Decision Tree Algorithm.

In [20]:
# importing the isloation forest
from sklearn.ensemble import IsolationForest
def apply_iso_forest(processed_df):
  processed_df = processed_df.copy()
  # copying dataset
  isf_dataset = processed_df.copy()

  # initializing Isolation Forest
  clf = IsolationForest(max_samples='auto', contamination=0.01)

  # training
  clf.fit(isf_dataset)

  # finding anomalies
  isf_dataset['Anomaly'] = clf.predict(isf_dataset)

  # saving anomalies to a separate dataset for visualization purposes
  anomalies = isf_dataset.query('Anomaly == -1')
  print(f"Number of Anomalies {len(anomalies)}")

  return isf_dataset, anomalies


In [21]:
plot_anomalies(*apply_iso_forest(processed_select_df), "Isolation Forest")

Number of Anomalies 150


In [22]:
plot_anomalies(*apply_iso_forest(processed_df), "Isolation Forest")

Number of Anomalies 150


# Local Outlier Factor (LOF)

The Local Outlier Factor (LOF) algorithm helps identify outliers based on the density of data points for every local data point in the dataset. The algorithm performs well when the data density is not the same throughout the dataset.


In [23]:
# Importing then local outlier factor
from sklearn.neighbors import LocalOutlierFactor
def apply_lof(processed_df):
  # copying dataset
  lof_dataset = processed_df.copy()

  # initializing the Local Outlier Factor algorithm
  clf = LocalOutlierFactor(n_neighbors=10)

  # training and finding anomalies
  lof_dataset['Anomaly'] = clf.fit_predict(lof_dataset)

  # saving anomalies to another dataset for visualization purposes
  anomalies = lof_dataset.query('Anomaly == -1')
  print(f"Number of Anomalies {len(anomalies)}")
  return lof_dataset, anomalies

In [24]:
plot_anomalies(*apply_lof(processed_df), "Local Outlier Factor")

Number of Anomalies 511


In [25]:
plot_anomalies(*apply_lof(processed_select_df), "Local Outlier Factor")

Number of Anomalies 511


In [26]:
plot_anomalies(*apply_lof(processed_select_df2), "Local Outlier Factor")

Number of Anomalies 563


Data features selection we got 150 anomalies with Isolation Forest.
Data w/o consatnt features selection we got 511 anomalies with Isolation Forest. <br>
Data without quasi-consatnt we got 563 anomalies with Local Outlier Factor.
